In [160]:
import pandas as pd
import re

In [161]:
df_radiology = pd.read_csv("radiology.csv")

In [162]:
pd.set_option('display.max_colwidth', None)

In [163]:
df_radiology.head(5)

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-RR-14,10000032,22595853.0,RR,14,2180-05-06 21:19:00,2180-05-06 23:32:00,"EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION: ___ with new onset ascites // eval for infection\n\nTECHNIQUE: Chest PA and lateral\n\nCOMPARISON: None.\n\nFINDINGS: \n\nThere is no focal consolidation, pleural effusion or pneumothorax. Bilateral\nnodular opacities that most likely represent nipple shadows. The\ncardiomediastinal silhouette is normal. Clips project over the left lung,\npotentially within the breast. The imaged upper abdomen is unremarkable.\nChronic deformity of the posterior left sixth and seventh ribs are noted.\n\nIMPRESSION: \n\nNo acute cardiopulmonary process.\n"
1,10000032-RR-15,10000032,22595853.0,RR,15,2180-05-06 23:00:00,2180-05-06 23:26:00,"EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ORGAN)\n\nINDICATION: ___ year-old female with cirrhosis, jaundice.\n\nTECHNIQUE: Grey scale and color Doppler ultrasound images of the abdomen were\nobtained.\n\nCOMPARISON: None.\n\nFINDINGS: \n\nLIVER: The liver is coarsened and nodular in echotexture. There is no focal\nliver mass. Main portal vein and its major branches are patent with normal\nhepatopetal flow. The main hepatic artery shows normal arterial waveform. \nThere is a small amount of ascites.\n\nBILE DUCTS: There is no intrahepatic biliary dilation. The CBD measures 4 mm.\n\nGALLBLADDER: The gallbladder is contracted with a shadowing gallstone.\n\nPANCREAS: Imaged portion of the pancreas appears within normal limits, without\nmasses or pancreatic ductal dilation, with portions of the pancreatic tail\nobscured by overlying bowel gas.\n\nSPLEEN: Normal echogenicity, measuring 13.5 cm.\n\nKIDNEYS: The right kidney measures 12.1 cm. The left kidney measures 13.4 cm.\nNormal cortical echogenicity and corticomedullary differentiation is seen\nbilaterally.\nRETROPERITONEUM: Visualized portions of aorta and IVC are within normal\nlimits.\n\nIMPRESSION: \n\n1. Nodular appearance of the liver compatible with cirrhosis. Signs of portal\nhypertension including small amount of ascites and splenomegaly.\n2. Cholelithiasis.\n3. Patent portal veins with normal hepatopetal flow.\n"
2,10000032-RR-16,10000032,22595853.0,RR,16,2180-05-07 09:55:00,2180-05-07 11:15:00,"INDICATION: ___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD,\nbioplar, PTSD, presented from OSH ED with worsening abd distension over past\nweek. // SBP\n\nTECHNIQUE: Ultrasound guided diagnostic and therapeutic paracentesis\n\nCOMPARISON: Abdominal ultrasound ___\n\nFINDINGS: \n\nLimited grayscale ultrasound imaging of the abdomen demonstrated\nmoderateascites. A suitable target in the deepest pocket in the right lower\nquadrant was selected for paracentesis.\n\nPROCEDURE: The procedure, risks, benefits and alternatives were discussed\nwith the patient and written informed consent was obtained.\n\nA preprocedure time-out was performed discussing the planned procedure,\nconfirming the patient's identity with 3 identifiers, and reviewing a\nchecklist per ___ protocol.\n\nUnder ultrasound guidance, an entrance site was selected and the skin was\nprepped and draped in the usual sterile fashion. 1% lidocaine was instilled\nfor local anesthesia.\n\nA 5 ___ catheter was advanced into the largest fluid pocket in the right\nlower quadrant and 1.5 L of serosanguinous fluid was removed.\n\nThe patient tolerated the procedure well without immediate complication.\nEstimated blood loss was minimal. A sample of the fluid was sent to the lab\nas requested.\n\nDr. ___ attending radiologist, was present throughout the critical\nportions of the procedure.\n\nIMPRESSION: \n\nSuccessful uncomplicated ultrasound guided diagnostic and therapeutic\nparacentesis yielding 1.5 L of serosanguineous fluid from the right lower\nquadrant. Sample was sent to the lab as requested.\n"
3,10000032-RR-18,10000032,NaN,RR,18,2180-06-03 12:46:00,2180-06-03 14:01:00,"EXAMINATION: Ultraso

In [164]:
# looking for missing values
missing_values = df_radiology.isnull()
missing_counts = missing_values.sum()
missing_counts

note_id             0
subject_id          0
hadm_id       1176597
note_type           0
note_seq            0
charttime           0
storetime           0
text                0
dtype: int64

In [165]:
# removing missing rows
df_radiology_clean = df_radiology.dropna()

In [166]:
df_radiology_clean

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-RR-14,10000032,22595853.0,RR,14,2180-05-06 21:19:00,2180-05-06 23:32:00,"EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION: ___ with new onset ascites // eval for infection\n\nTECHNIQUE: Chest PA and lateral\n\nCOMPARISON: None.\n\nFINDINGS: \n\nThere is no focal consolidation, pleural effusion or pneumothorax. Bilateral\nnodular opacities that most likely represent nipple shadows. The\ncardiomediastinal silhouette is normal. Clips project over the left lung,\npotentially within the breast. The imaged upper abdomen is unremarkable.\nChronic deformity of the posterior left sixth and seventh ribs are noted.\n\nIMPRESSION: \n\nNo acute cardiopulmonary process.\n"
1,10000032-RR-15,10000032,22595853.0,RR,15,2180-05-06 23:00:00,2180-05-06 23:26:00,"EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ORGAN)\n\nINDICATION: ___ year-old female with cirrhosis, jaundice.\n\nTECHNIQUE: Grey scale and color Doppler ultrasound images of the abdomen were\nobtained.\n\nCOMPARISON: None.\n\nFINDINGS: \n\nLIVER: The liver is coarsened and nodular in echotexture. There is no focal\nliver mass. Main portal vein and its major branches are patent with normal\nhepatopetal flow. The main hepatic artery shows normal arterial waveform. \nThere is a small amount of ascites.\n\nBILE DUCTS: There is no intrahepatic biliary dilation. The CBD measures 4 mm.\n\nGALLBLADDER: The gallbladder is contracted with a shadowing gallstone.\n\nPANCREAS: Imaged portion of the pancreas appears within normal limits, without\nmasses or pancreatic ductal dilation, with portions of the pancreatic tail\nobscured by overlying bowel gas.\n\nSPLEEN: Normal echogenicity, measuring 13.5 cm.\n\nKIDNEYS: The right kidney measures 12.1 cm. The left kidney measures 13.4 cm.\nNormal cortical echogenicity and corticomedullary differentiation is seen\nbilaterally.\nRETROPERITONEUM: Visualized portions of aorta and IVC are within normal\nlimits.\n\nIMPRESSION: \n\n1. Nodular appearance of the liver compatible with cirrhosis. Signs of portal\nhypertension including small amount of ascites and splenomegaly.\n2. Cholelithiasis.\n3. Patent portal veins with normal hepatopetal flow.\n"
2,10000032-RR-16,10000032,22595853.0,RR,16,2180-05-07 09:55:00,2180-05-07 11:15:00,"INDICATION: ___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD,\nbioplar, PTSD, presented from OSH ED with worsening abd distension over past\nweek. // SBP\n\nTECHNIQUE: Ultrasound guided diagnostic and therapeutic paracentesis\n\nCOMPARISON: Abdominal ultrasound ___\n\nFINDINGS: \n\nLimited grayscale ultrasound imaging of the abdomen demonstrated\nmoderateascites. A suitable target in the deepest pocket in the right lower\nquadrant was selected for paracentesis.\n\nPROCEDURE: The procedure, risks, benefits and alternatives were discussed\nwith the patient and written informed consent was obtained.\n\nA preprocedure time-out was performed discussing the planned procedure,\nconfirming the patient's identity with 3 identifiers, and reviewing a\nchecklist per ___ protocol.\n\nUnder ultrasound guidance, an entrance site was selected and the skin was\nprepped and draped in the usual sterile fashion. 1% lidocaine was instilled\nfor local anesthesia.\n\nA 5 ___ catheter was advanced into the largest fluid pocket in the right\nlower quadrant and 1.5 L of serosanguinous fluid was removed.\n\nThe patient tolerated the procedure well without immediate complication.\nEstimated blood loss was minimal. A sample of the fluid was sent to the lab\nas requested.\n\nDr. ___ attending radiologist, was present throughout the critical\nportions of the procedure.\n\nIMPRESSION: \n\nSuccessful uncomplicated ultrasound guided diagnostic and therapeutic\nparacentesis yielding 1.5 L of serosanguineous fluid from the right lower\nquadrant. Sample was sent to the lab as requested.\n"
6,10000032-RR-22,10000032,22841357.0,RR,22,2180-06-26 17:15:00,2180-06-26 19:28:00,"EXAMINATION: 

In [167]:
print(df_radiology_clean.shape)

(1144758, 8)


In [168]:
# taking a subset of data
subset = df_radiology_clean.iloc[:50000]

In [169]:
subset.shape

(50000, 8)

In [170]:
subset.head(5)

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-RR-14,10000032,22595853.0,RR,14,2180-05-06 21:19:00,2180-05-06 23:32:00,"EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION: ___ with new onset ascites // eval for infection\n\nTECHNIQUE: Chest PA and lateral\n\nCOMPARISON: None.\n\nFINDINGS: \n\nThere is no focal consolidation, pleural effusion or pneumothorax. Bilateral\nnodular opacities that most likely represent nipple shadows. The\ncardiomediastinal silhouette is normal. Clips project over the left lung,\npotentially within the breast. The imaged upper abdomen is unremarkable.\nChronic deformity of the posterior left sixth and seventh ribs are noted.\n\nIMPRESSION: \n\nNo acute cardiopulmonary process.\n"
1,10000032-RR-15,10000032,22595853.0,RR,15,2180-05-06 23:00:00,2180-05-06 23:26:00,"EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ORGAN)\n\nINDICATION: ___ year-old female with cirrhosis, jaundice.\n\nTECHNIQUE: Grey scale and color Doppler ultrasound images of the abdomen were\nobtained.\n\nCOMPARISON: None.\n\nFINDINGS: \n\nLIVER: The liver is coarsened and nodular in echotexture. There is no focal\nliver mass. Main portal vein and its major branches are patent with normal\nhepatopetal flow. The main hepatic artery shows normal arterial waveform. \nThere is a small amount of ascites.\n\nBILE DUCTS: There is no intrahepatic biliary dilation. The CBD measures 4 mm.\n\nGALLBLADDER: The gallbladder is contracted with a shadowing gallstone.\n\nPANCREAS: Imaged portion of the pancreas appears within normal limits, without\nmasses or pancreatic ductal dilation, with portions of the pancreatic tail\nobscured by overlying bowel gas.\n\nSPLEEN: Normal echogenicity, measuring 13.5 cm.\n\nKIDNEYS: The right kidney measures 12.1 cm. The left kidney measures 13.4 cm.\nNormal cortical echogenicity and corticomedullary differentiation is seen\nbilaterally.\nRETROPERITONEUM: Visualized portions of aorta and IVC are within normal\nlimits.\n\nIMPRESSION: \n\n1. Nodular appearance of the liver compatible with cirrhosis. Signs of portal\nhypertension including small amount of ascites and splenomegaly.\n2. Cholelithiasis.\n3. Patent portal veins with normal hepatopetal flow.\n"
2,10000032-RR-16,10000032,22595853.0,RR,16,2180-05-07 09:55:00,2180-05-07 11:15:00,"INDICATION: ___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD,\nbioplar, PTSD, presented from OSH ED with worsening abd distension over past\nweek. // SBP\n\nTECHNIQUE: Ultrasound guided diagnostic and therapeutic paracentesis\n\nCOMPARISON: Abdominal ultrasound ___\n\nFINDINGS: \n\nLimited grayscale ultrasound imaging of the abdomen demonstrated\nmoderateascites. A suitable target in the deepest pocket in the right lower\nquadrant was selected for paracentesis.\n\nPROCEDURE: The procedure, risks, benefits and alternatives were discussed\nwith the patient and written informed consent was obtained.\n\nA preprocedure time-out was performed discussing the planned procedure,\nconfirming the patient's identity with 3 identifiers, and reviewing a\nchecklist per ___ protocol.\n\nUnder ultrasound guidance, an entrance site was selected and the skin was\nprepped and draped in the usual sterile fashion. 1% lidocaine was instilled\nfor local anesthesia.\n\nA 5 ___ catheter was advanced into the largest fluid pocket in the right\nlower quadrant and 1.5 L of serosanguinous fluid was removed.\n\nThe patient tolerated the procedure well without immediate complication.\nEstimated blood loss was minimal. A sample of the fluid was sent to the lab\nas requested.\n\nDr. ___ attending radiologist, was present throughout the critical\nportions of the procedure.\n\nIMPRESSION: \n\nSuccessful uncomplicated ultrasound guided diagnostic and therapeutic\nparacentesis yielding 1.5 L of serosanguineous fluid from the right lower\nquadrant. Sample was sent to the lab as requested.\n"
6,10000032-RR-22,10000032,22841357.0,RR,22,2180-06-26 17:15:00,2180-06-26 19:28:00,"EXAMINATION: 

In [171]:
# replacing all occurrences of "\n" with " " 
subset.loc[:, "text"] = subset["text"].str.replace("\n", " ")
subset.head(5)

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-RR-14,10000032,22595853.0,RR,14,2180-05-06 21:19:00,2180-05-06 23:32:00,"EXAMINATION: CHEST (PA AND LAT) INDICATION: ___ with new onset ascites // eval for infection TECHNIQUE: Chest PA and lateral COMPARISON: None. FINDINGS: There is no focal consolidation, pleural effusion or pneumothorax. Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal. Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted. IMPRESSION: No acute cardiopulmonary process."
1,10000032-RR-15,10000032,22595853.0,RR,15,2180-05-06 23:00:00,2180-05-06 23:26:00,"EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ORGAN) INDICATION: ___ year-old female with cirrhosis, jaundice. TECHNIQUE: Grey scale and color Doppler ultrasound images of the abdomen were obtained. COMPARISON: None. FINDINGS: LIVER: The liver is coarsened and nodular in echotexture. There is no focal liver mass. Main portal vein and its major branches are patent with normal hepatopetal flow. The main hepatic artery shows normal arterial waveform. There is a small amount of ascites. BILE DUCTS: There is no intrahepatic biliary dilation. The CBD measures 4 mm. GALLBLADDER: The gallbladder is contracted with a shadowing gallstone. PANCREAS: Imaged portion of the pancreas appears within normal limits, without masses or pancreatic ductal dilation, with portions of the pancreatic tail obscured by overlying bowel gas. SPLEEN: Normal echogenicity, measuring 13.5 cm. KIDNEYS: The right kidney measures 12.1 cm. The left kidney measures 13.4 cm. Normal cortical echogenicity and corticomedullary differentiation is seen bilaterally. RETROPERITONEUM: Visualized portions of aorta and IVC are within normal limits. IMPRESSION: 1. Nodular appearance of the liver compatible with cirrhosis. Signs of portal hypertension including small amount of ascites and splenomegaly. 2. Cholelithiasis. 3. Patent portal veins with normal hepatopetal flow."
2,10000032-RR-16,10000032,22595853.0,RR,16,2180-05-07 09:55:00,2180-05-07 11:15:00,"INDICATION: ___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. // SBP TECHNIQUE: Ultrasound guided diagnostic and therapeutic paracentesis COMPARISON: Abdominal ultrasound ___ FINDINGS: Limited grayscale ultrasound imaging of the abdomen demonstrated moderateascites. A suitable target in the deepest pocket in the right lower quadrant was selected for paracentesis. PROCEDURE: The procedure, risks, benefits and alternatives were discussed with the patient and written informed consent was obtained. A preprocedure time-out was performed discussing the planned procedure, confirming the patient's identity with 3 identifiers, and reviewing a checklist per ___ protocol. Under ultrasound guidance, an entrance site was selected and the skin was prepped and draped in the usual sterile fashion. 1% lidocaine was instilled for local anesthesia. A 5 ___ catheter was advanced into the largest fluid pocket in the right lower quadrant and 1.5 L of serosanguinous fluid was removed. The patient tolerated the procedure well without immediate complication. Estimated blood loss was minimal. A sample of the fluid was sent to the lab as requested. Dr. ___ attending radiologist, was present throughout the critical portions of the procedure. IMPRESSION: Successful uncomplicated ultrasound guided diagnostic and therapeutic paracentesis yielding 1.5 L of serosanguineous fluid from the right lower quadrant. Sample was sent to the lab as requested."
6,10000032-RR-22,10000032,22841357.0,RR,22,2180-06-26 17:15:00,2180-06-26 19:28:00,"EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ORGAN) INDICATION: History: ___ with cirrhosis, increased abdominal pain TECHNIQUE: Grey scale and color D

In [172]:
text_subset = subset[["text"]]
text_subset.head(5)

,text
0,"EXAMINATION: CHEST (PA AND LAT) INDICATION: ___ with new onset ascites // eval for infection TECHNIQUE: Chest PA and lateral COMPARISON: None. FINDINGS: There is no focal consolidation, pleural effusion or pneumothorax. Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal. Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted. IMPRESSION: No acute cardiopulmonary process."
1,"EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ORGAN) INDICATION: ___ year-old female with cirrhosis, jaundice. TECHNIQUE: Grey scale and color Doppler ultrasound images of the abdomen were obtained. COMPARISON: None. FINDINGS: LIVER: The liver is coarsened and nodular in echotexture. There is no focal liver mass. Main portal vein and its major branches are patent with normal hepatopetal flow. The main hepatic artery shows normal arterial waveform. There is a small amount of ascites. BILE DUCTS: There is no intrahepatic biliary dilation. The CBD measures 4 mm. GALLBLADDER: The gallbladder is contracted with a shadowing gallstone. PANCREAS: Imaged portion of the pancreas appears within normal limits, without masses or pancreatic ductal dilation, with portions of the pancreatic tail obscured by overlying bowel gas. SPLEEN: Normal echogenicity, measuring 13.5 cm. KIDNEYS: The right kidney measures 12.1 cm. The left kidney measures 13.4 cm. Normal cortical echogenicity and corticomedullary differentiation is seen bilaterally. RETROPERITONEUM: Visualized portions of aorta and IVC are within normal limits. IMPRESSION: 1. Nodular appearance of the liver compatible with cirrhosis. Signs of portal hypertension including small amount of ascites and splenomegaly. 2. Cholelithiasis. 3. Patent portal veins with normal hepatopetal flow."
2,"INDICATION: ___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. // SBP TECHNIQUE: Ultrasound guided diagnostic and therapeutic paracentesis COMPARISON: Abdominal ultrasound ___ FINDINGS: Limited grayscale ultrasound imaging of the abdomen demonstrated moderateascites. A suitable target in the deepest pocket in the right lower quadrant was selected for paracentesis. PROCEDURE: The procedure, risks, benefits and alternatives were discussed with the patient and written informed consent was obtained. A preprocedure time-out was performed discussing the planned procedure, confirming the patient's identity with 3 identifiers, and reviewing a checklist per ___ protocol. Under ultrasound guidance, an entrance site was selected and the skin was prepped and draped in the usual sterile fashion. 1% lidocaine was instilled for local anesthesia. A 5 ___ catheter was advanced into the largest fluid pocket in the right lower quadrant and 1.5 L of serosanguinous fluid was removed. The patient tolerated the procedure well without immediate complication. Estimated blood loss was minimal. A sample of the fluid was sent to the lab as requested. Dr. ___ attending radiologist, was present throughout the critical portions of the procedure. IMPRESSION: Successful uncomplicated ultrasound guided diagnostic and therapeutic paracentesis yielding 1.5 L of serosanguineous fluid from the right lower quadrant. Sample was sent to the lab as requested."
6,"EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ORGAN) INDICATION: History: ___ with cirrhosis, increased abdominal pain TECHNIQUE: Grey scale and color Doppler ultrasound images of the right upper quadrant were obtained. COMPARISON: Abdominal ultrasound from ___ FINDINGS: The liver is extremely course and nodular in echotexture similar to the prior examination consistent with a history of cirrhosis. Parenchymal heterogeneity limits detection focal lesions. Note is made of a dominant nodule measuring 3.3 x 2.7 cm exerting mass effect on t

In [173]:
# importing tokenizer lib
import nltk

# downloading Punkt tokenizer 
nltk.download("punkt")


[nltk_data] Downloading package punkt to /Users/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [174]:
from nltk import word_tokenize

# convert all letters to lowercase
text_subset.loc[:, "text"] = text_subset["text"].str.lower()
# remove all occurrences of ___ blnk thing
text_subset.loc[:, "text"] = text_subset["text"].str.replace("___", "")
text_subset.head(5)

,text
0,"examination: chest (pa and lat) indication: with new onset ascites // eval for infection technique: chest pa and lateral comparison: none. findings: there is no focal consolidation, pleural effusion or pneumothorax. bilateral nodular opacities that most likely represent nipple shadows. the cardiomediastinal silhouette is normal. clips project over the left lung, potentially within the breast. the imaged upper abdomen is unremarkable. chronic deformity of the posterior left sixth and seventh ribs are noted. impression: no acute cardiopulmonary process."
1,"examination: liver or gallbladder us (single organ) indication: year-old female with cirrhosis, jaundice. technique: grey scale and color doppler ultrasound images of the abdomen were obtained. comparison: none. findings: liver: the liver is coarsened and nodular in echotexture. there is no focal liver mass. main portal vein and its major branches are patent with normal hepatopetal flow. the main hepatic artery shows normal arterial waveform. there is a small amount of ascites. bile ducts: there is no intrahepatic biliary dilation. the cbd measures 4 mm. gallbladder: the gallbladder is contracted with a shadowing gallstone. pancreas: imaged portion of the pancreas appears within normal limits, without masses or pancreatic ductal dilation, with portions of the pancreatic tail obscured by overlying bowel gas. spleen: normal echogenicity, measuring 13.5 cm. kidneys: the right kidney measures 12.1 cm. the left kidney measures 13.4 cm. normal cortical echogenicity and corticomedullary differentiation is seen bilaterally. retroperitoneum: visualized portions of aorta and ivc are within normal limits. impression: 1. nodular appearance of the liver compatible with cirrhosis. signs of portal hypertension including small amount of ascites and splenomegaly. 2. cholelithiasis. 3. patent portal veins with normal hepatopetal flow."
2,"indication: hcv cirrhosis c/b ascites, hiv on art, h/o ivdu, copd, bioplar, ptsd, presented from osh ed with worsening abd distension over past week. // sbp technique: ultrasound guided diagnostic and therapeutic paracentesis comparison: abdominal ultrasound findings: limited grayscale ultrasound imaging of the abdomen demonstrated moderateascites. a suitable target in the deepest pocket in the right lower quadrant was selected for paracentesis. procedure: the procedure, risks, benefits and alternatives were discussed with the patient and written informed consent was obtained. a preprocedure time-out was performed discussing the planned procedure, confirming the patient's identity with 3 identifiers, and reviewing a checklist per protocol. under ultrasound guidance, an entrance site was selected and the skin was prepped and draped in the usual sterile fashion. 1% lidocaine was instilled for local anesthesia. a 5 catheter was advanced into the largest fluid pocket in the right lower quadrant and 1.5 l of serosanguinous fluid was removed. the patient tolerated the procedure well without immediate complication. estimated blood loss was minimal. a sample of the fluid was sent to the lab as requested. dr. attending radiologist, was present throughout the critical portions of the procedure. impression: successful uncomplicated ultrasound guided diagnostic and therapeutic paracentesis yielding 1.5 l of serosanguineous fluid from the right lower quadrant. sample was sent to the lab as requested."
6,"examination: liver or gallbladder us (single organ) indication: history: with cirrhosis, increased abdominal pain technique: grey scale and color doppler ultrasound images of the right upper quadrant were obtained. comparison: abdominal ultrasound from findings: the liver is extremely course and nodular in echotexture similar to the prior examination consistent with a history of cirrhosis. parenchymal heterogeneity limits detection focal lesions. note is made of a dominant nodule measuring 3.3 x 2.7 cm exerting mass effect on the gallbladder, relatively unchanged

In [175]:
# removing numbers from text
text_subset.loc[:, "text"] = text_subset["text"].str.replace(r'\d+', '', regex=True)
text_subset.loc[:, "text"] = text_subset["text"].str.replace(' .', '.')
text_subset.head(5)

,text
0,"examination: chest (pa and lat) indication: with new onset ascites // eval for infection technique: chest pa and lateral comparison: none. findings: there is no focal consolidation, pleural effusion or pneumothorax. bilateral nodular opacities that most likely represent nipple shadows. the cardiomediastinal silhouette is normal. clips project over the left lung, potentially within the breast. the imaged upper abdomen is unremarkable. chronic deformity of the posterior left sixth and seventh ribs are noted. impression: no acute cardiopulmonary process."
1,"examination: liver or gallbladder us (single organ) indication: year-old female with cirrhosis, jaundice. technique: grey scale and color doppler ultrasound images of the abdomen were obtained. comparison: none. findings: liver: the liver is coarsened and nodular in echotexture. there is no focal liver mass. main portal vein and its major branches are patent with normal hepatopetal flow. the main hepatic artery shows normal arterial waveform. there is a small amount of ascites. bile ducts: there is no intrahepatic biliary dilation. the cbd measures mm. gallbladder: the gallbladder is contracted with a shadowing gallstone. pancreas: imaged portion of the pancreas appears within normal limits, without masses or pancreatic ductal dilation, with portions of the pancreatic tail obscured by overlying bowel gas. spleen: normal echogenicity, measuring. cm. kidneys: the right kidney measures. cm. the left kidney measures. cm. normal cortical echogenicity and corticomedullary differentiation is seen bilaterally. retroperitoneum: visualized portions of aorta and ivc are within normal limits. impression: . nodular appearance of the liver compatible with cirrhosis. signs of portal hypertension including small amount of ascites and splenomegaly.. cholelithiasis.. patent portal veins with normal hepatopetal flow."
2,"indication: hcv cirrhosis c/b ascites, hiv on art, h/o ivdu, copd, bioplar, ptsd, presented from osh ed with worsening abd distension over past week. // sbp technique: ultrasound guided diagnostic and therapeutic paracentesis comparison: abdominal ultrasound findings: limited grayscale ultrasound imaging of the abdomen demonstrated moderateascites. a suitable target in the deepest pocket in the right lower quadrant was selected for paracentesis. procedure: the procedure, risks, benefits and alternatives were discussed with the patient and written informed consent was obtained. a preprocedure time-out was performed discussing the planned procedure, confirming the patient's identity with identifiers, and reviewing a checklist per protocol. under ultrasound guidance, an entrance site was selected and the skin was prepped and draped in the usual sterile fashion. % lidocaine was instilled for local anesthesia. a catheter was advanced into the largest fluid pocket in the right lower quadrant and. l of serosanguinous fluid was removed. the patient tolerated the procedure well without immediate complication. estimated blood loss was minimal. a sample of the fluid was sent to the lab as requested. dr. attending radiologist, was present throughout the critical portions of the procedure. impression: successful uncomplicated ultrasound guided diagnostic and therapeutic paracentesis yielding. l of serosanguineous fluid from the right lower quadrant. sample was sent to the lab as requested."
6,"examination: liver or gallbladder us (single organ) indication: history: with cirrhosis, increased abdominal pain technique: grey scale and color doppler ultrasound images of the right upper quadrant were obtained. comparison: abdominal ultrasound from findings: the liver is extremely course and nodular in echotexture similar to the prior examination consistent with a history of cirrhosis. parenchymal heterogeneity limits detection focal lesions. note is made of a dominant nodule measuring. x. cm exerting mass effect on the gallbladder, relatively unchanged from the prior examination. there i

In [176]:
import string

# # removing punctuation
# def remove_punctuation(text):
#     return text.str.replace('[{}]'.format(string.punctuation), '', regex = True)

# # removing punctuation except for periods
# def remove_punctuation(text):
#     # define pattern match
#     punctuation_pattern = r'[^\w\s.]'
#     cleaned_text = re.sub(punctuation_pattern, "", text)
#     return cleaned_text

# text_subset.loc[:, "text"] = remove_punctuation(text_subset['text'])
def remove_punctuation_except_periods(text):
    # Define a pattern to match punctuation (excluding periods)
    punctuation_pattern = r'[^\w\s.]'
    # Use re.sub to remove punctuation
    cleaned_text = re.sub(punctuation_pattern, '', text)
    return cleaned_text

text_subset.loc[:, "text"] = text_subset["text"].apply(remove_punctuation_except_periods)
text_subset.head(5)

,text
0,examination chest pa and lat indication with new onset ascites eval for infection technique chest pa and lateral comparison none. findings there is no focal consolidation pleural effusion or pneumothorax. bilateral nodular opacities that most likely represent nipple shadows. the cardiomediastinal silhouette is normal. clips project over the left lung potentially within the breast. the imaged upper abdomen is unremarkable. chronic deformity of the posterior left sixth and seventh ribs are noted. impression no acute cardiopulmonary process.
1,examination liver or gallbladder us single organ indication yearold female with cirrhosis jaundice. technique grey scale and color doppler ultrasound images of the abdomen were obtained. comparison none. findings liver the liver is coarsened and nodular in echotexture. there is no focal liver mass. main portal vein and its major branches are patent with normal hepatopetal flow. the main hepatic artery shows normal arterial waveform. there is a small amount of ascites. bile ducts there is no intrahepatic biliary dilation. the cbd measures mm. gallbladder the gallbladder is contracted with a shadowing gallstone. pancreas imaged portion of the pancreas appears within normal limits without masses or pancreatic ductal dilation with portions of the pancreatic tail obscured by overlying bowel gas. spleen normal echogenicity measuring. cm. kidneys the right kidney measures. cm. the left kidney measures. cm. normal cortical echogenicity and corticomedullary differentiation is seen bilaterally. retroperitoneum visualized portions of aorta and ivc are within normal limits. impression . nodular appearance of the liver compatible with cirrhosis. signs of portal hypertension including small amount of ascites and splenomegaly.. cholelithiasis.. patent portal veins with normal hepatopetal flow.
2,indication hcv cirrhosis cb ascites hiv on art ho ivdu copd bioplar ptsd presented from osh ed with worsening abd distension over past week. sbp technique ultrasound guided diagnostic and therapeutic paracentesis comparison abdominal ultrasound findings limited grayscale ultrasound imaging of the abdomen demonstrated moderateascites. a suitable target in the deepest pocket in the right lower quadrant was selected for paracentesis. procedure the procedure risks benefits and alternatives were discussed with the patient and written informed consent was obtained. a preprocedure timeout was performed discussing the planned procedure confirming the patients identity with identifiers and reviewing a checklist per protocol. under ultrasound guidance an entrance site was selected and the skin was prepped and draped in the usual sterile fashion. lidocaine was instilled for local anesthesia. a catheter was advanced into the largest fluid pocket in the right lower quadrant and. l of serosanguinous fluid was removed. the patient tolerated the procedure well without immediate complication. estimated blood loss was minimal. a sample of the fluid was sent to the lab as requested. dr. attending radiologist was present throughout the critical portions of the procedure. impression successful uncomplicated ultrasound guided diagnostic and therapeutic paracentesis yielding. l of serosanguineous fluid from the right lower quadrant. sample was sent to the lab as requested.
6,examination liver or gallbladder us single organ indication history with cirrhosis increased abdominal pain technique grey scale and color doppler ultrasound images of the right upper quadrant were obtained. comparison abdominal ultrasound from findings the liver is extremely course and nodular in echotexture similar to the prior examination consistent with a history of cirrhosis. parenchymal heterogeneity limits detection focal lesions. note is made of a dominant nodule measuring. x. cm exerting mass effect on the gallbladder relatively unchanged from the prior examination. there is no intrahepatic biliary dilation. the cbd measures mm. main right anterior 

In [177]:
# executing lemmatisation before tokenisation
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# download WordNet data 
nltk.download("wordnet")

lemmatiser = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /Users/alex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [178]:
# WARNING: == using nltk takes too long, use spaCy instead ==
# # create function to lemmatise
# def lemmatise_text(text):
#     words = nltk.word_tokenize(text)
#     lemmatised_words = [lemmatiser.lemmatize(word, wordnet.VERB) for word in words]
#     return " ".join(lemmatised_words)

# # apply func to text col
# text_subset.loc[:, "text"] = text_subset["text"].apply(lemmatise_text)
# text_subset.head(5)

In [179]:
# importing spacy 
import spacy 
# load englang model
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

In [180]:
if "sentencizer" not in nlp.pipe_names:
    # sentencizer = nlp.create_pipe("sentencizer")
    nlp.add_pipe("sentencizer")

def lemmatize_text(text):
    # using batch lemmatisationg for efficiency
    doc = nlp(text)
    lemmatized_words = [" ".join([token.lemma_ for token in sentence]) for sentence in doc.sents]
    return " ".join(lemmatized_words)

chunk_size = 1000
num_chunks = len(text_subset) // (chunk_size + 1)

for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk = text_subset.iloc[start_idx:end_idx].copy()

    chunk["text"] = chunk["text"].str.lower()

    # lemmatise
    chunk["text"] = chunk["text"].apply(lemmatize_text)

    # update text_subset
    text_subset.iloc[start_idx: end_idx] = chunk

# apply func to text col
# text_subset.loc[:, "text"] = text_subset["text"].apply(lemmatize_text)
text_subset.head(5)

/var/folders/pc/77rl9hn521l6g12xys12l4br0000gp/T/ipykernel_30662/4267929109.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_subset.iloc[start_idx: end_idx] = chunk
/var/folders/pc/77rl9hn521l6g12xys12l4br0000gp/T/ipykernel_30662/4267929109.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_subset.iloc[start_idx: end_idx] = chunk
/var/folders/pc/77rl9hn521l6g12xys12l4br0000gp/T/ipykernel_30662/4267929109.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

KeyboardInterrupt: 